## Creating snowflake connection and pushing data 

In [9]:
from sqlalchemy import create_engine
import pandas as pd


engine = create_engine(
    'snowflake://{user}:{password}@{account}/'.format(
        user='GEORGE9042',
        password='George9042',
        account='plninim-tg58176',
        warehouse='COMPUTE_WH',
        database='MY_TEST', 
        schema='NEW'
    )
)
sfOptions = {
    "sfURL": "https://lv50176.switzerland-north.azure.snowflakecomputing.com",
    "sfAccount": "plninim-tg58176",
    "sfUser": "GEORGE9042",
    "sfPassword": "George9042",
    "sfDatabase": "MY_TEST",
    "sfSchema": "NEW",
    "sfWarehouse": "COMPUTE_WH",
}
try:
    connection = engine.connect()
    connection.execute('USE ROLE ACCOUNTADMIN')
    connection.execute('USE DATABASE MY_TEST')
    connection.execute('USE SCHEMA NEW')

    #airports = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/airports.csv')
    #airports.to_sql('airports', con=connection, if_exists='replace',index = False,chunksize=16000)   

    #airport_frequencies = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/airport-frequencies.csv')
    #airport_frequencies.to_sql('airport_frequencies', con=connection, if_exists='replace',index = False,chunksize=16000) 

    #countries = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/countries.csv')
    #countries.to_sql('countries', con=connection, if_exists='replace',index = False,chunksize=16000)

    #airport_comments = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/airport-comments.csv')
    #airport_comments.to_sql('airport_comments', con=connection, if_exists='replace',index = False,chunksize=16000)

    #navaids = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/navaids.csv')
    #navaids.to_sql('navaids', con=connection, if_exists='replace',index = False,chunksize=16000)

    #regions = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/regions.csv')
    #regions.to_sql('regions', con=connection, if_exists='replace',index = False,chunksize=16000)

    #runways = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/runways.csv')
    #runways.to_sql('runways', con=connection, if_exists='replace',index = False,chunksize=16000)

#using Pyspark because the table has 12 million rows
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.appName("ReadTextFile").getOrCreate()
    header_names = ["geoname_id", "name", "asciiname","alternatenames","latitude","longitude",
    "feature_class","feature_code","country_code","altcountry_code","admin1_code","admin2_code"
    ,"admin3_code","admin4_code","population","elevation","DEM","timezone","modification_date" ]
    path = "/workspace/Airport_Pipeline/Geonames_data/allCountries.txt"


#text_file = spark.read.format("csv").option("delimiter", "\t").load("./Geonames_data/allCountries.txt")
    df = spark.read.option("header", "true").option("delimiter", "\t").csv(path).toDF(*header_names)
    # Specify the target table name in Snowflake
    pdf = df.toPandas()
    pdf.to_sql('geonames', con=connection, if_exists='replace',index = False,chunksize=16000)

finally:
    connection.close()
    engine.dispose()
    
    

23/05/28 17:52:19 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 2986043, Pic de Font Blanca, Pic de Font Blanca, Pic de Font Blanca,Pic du Port, 42.64991, 1.53335, T, PK, AD, , 00, , , , 0, , 2860, Europe/Andorra, 2014-11-05
 Schema: 2986043, Pic de Font Blanca1, Pic de Font Blanca2, Pic de Font Blanca,Pic du Port, 42.64991, 1.53335, T, PK, AD, _c9, 00, _c11, _c12, _c13, 0, _c15, 2860, Europe/Andorra, 2014-11-05
Expected: Pic de Font Blanca1 but found: Pic de Font Blanca
CSV file: file:///workspace/Airport_Pipeline/Geonames_data/allCountries.txt
23/05/28 17:52:35 ERROR Executor: Exception in task 7.0 in stage 11.0 (TID 18)6]
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:61)
	at java.base/java.nio.ByteBuffer.allocate(ByteBuffer.java:348)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$2(SparkPlan.scala:368)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRd

ConnectionRefusedError: [Errno 111] Connection refused

### 1. How many airports, airfields and heliports exist in each country and continent?

In [10]:
from sqlalchemy import create_engine
import pandas as pd


engine = create_engine(
    'snowflake://{user}:{password}@{account}/'.format(
        user='GEORGE9042',
        password='George9042',
        account='plninim-tg58176',
        warehouse='COMPUTE_WH',
        database='MY_TEST', 
        schema='NEW'
    )
)
try:
    connection = engine.connect()
    connection.execute('USE ROLE ACCOUNTADMIN')
    connection.execute('USE DATABASE MY_TEST')
    connection.execute('USE SCHEMA NEW')
    clean_continent = '''UPDATE airports SET CONTINENT = COALESCE(CONTINENT, 'NA')'''
    clean_country = '''UPDATE airports SET iso_country = COALESCE(iso_country, 'NA')'''

    query_continent = '''select * from airport_per_continent'''

    query_country='''select * from airport_per_country '''
    result = connection.execute(query_continent)
    for i in result:
             print(i)
finally:
    connection.close()
    engine.dispose()

('AN', 5, 9, 25)
('EU', 1065, 1714, 5679)
('SA', 434, 2152, 7312)
('NA', 2487, 8805, 17931)
('AF', 517, 209, 3056)
('AS', 1483, 5712, 2587)
('OC', 353, 479, 2829)


## Creating Snowflake Tables

In [ ]:
create or replace TABLE MY_TEST.NEW.AIRPORT_COMMENTS (
	ID NUMBER(38,0),
	threadRef FLOAT,
	airportRef NUMBER(38,0),
	airportIdent VARCHAR(16777216),
	date VARCHAR(16777216),
	memberNickname VARCHAR(16777216),
	subject VARCHAR(16777216),
	body VARCHAR(16777216),
    PRIMARY KEY (ID)
);

create or replace TABLE MY_TEST.NEW.AIRPORTS (
	ID NUMBER(38,0) NOT NULL,
	IDENT VARCHAR(16777216),
	TYPE VARCHAR(16777216),
	NAME VARCHAR(16777216),
	LATITUDE_DEG FLOAT,
	LONGITUDE_DEG FLOAT,
	ELEVATION_FT NUMBER(38,0),
	CONTINENT VARCHAR(16777216),
	ISO_COUNTRY VARCHAR(16777216),
	ISO_REGION VARCHAR(16777216),
	MUNICIPALITY VARCHAR(16777216),
	SCHEDULED_SERVICE VARCHAR(16777216),
	GPS_CODE VARCHAR(16777216),
	IATA_CODE VARCHAR(16777216),
	LOCAL_CODE VARCHAR(16777216),
	HOME_LINK VARCHAR(16777216),
	WIKIPEDIA_LINK VARCHAR(16777216),
	KEYWORDS VARCHAR(16777216),
	primary key (ID),
	foreign key (ISO_REGION) references MY_TEST.NEW.REGIONS(CODE)
);

create or replace TABLE MY_TEST.NEW.RUNWAYS (
	ID NUMBER(38,0) NOT NULL,
	AIRPORT_REF NUMBER(38,0),
	AIRPORT_IDENT VARCHAR(16777216),
	LENGTH_FT FLOAT,
	WIDTH_FT FLOAT,
	SURFACE VARCHAR(16777216),
	LIGHTED BOOLEAN,
	CLOSED BOOLEAN,
	LE_IDENT VARCHAR(16777216),
	LE_LATITUDE_DEG FLOAT,
	LE_LONGITUDE_DEG FLOAT,
	LE_ELEVATION_FT NUMBER(38,0),
	LE_HEADING_DEGT FLOAT,
	LE_DISPLACED_THRESHOLD_FT NUMBER(38,0),
	HE_IDENT VARCHAR(16777216),
	HE_LATITUDE_DEG FLOAT,
	HE_LONGITUDE_DEG FLOAT,
	HE_ELEVATION_FT NUMBER(38,0),
	HE_HEADING_DEGT FLOAT,
	HE_DISPLACED_THRESHOLD_FT NUMBER(38,0),
	primary key (ID),
	foreign key (AIRPORT_REF) references MY_TEST.NEW.AIRPORTS(ID)
);

create or replace TABLE MY_TEST.NEW.REGIONS (
	CODE VARCHAR(16777216) NOT NULL,
	LOCAL_CODE VARCHAR(16777216),
	ISO_COUNTRY VARCHAR(16777216),
	NAME VARCHAR(16777216),
	CONTINENT VARCHAR(16777216),
	primary key (CODE)
);

create or replace TABLE MY_TEST.NEW.NAVAIDS (
	ID_NAV NUMBER(38,0) NOT NULL,
	FILENAME VARCHAR(16777216),
	IDENT VARCHAR(16777216),
	NAME VARCHAR(16777216),
	TYPE VARCHAR(16777216),
	FREQUENCY_KHZ FLOAT,
	LATITUDE_DEG FLOAT,
	LONGITUDE_DEG FLOAT,
	ELEVATION_FT NUMBER(38,0),
	ISO_COUNTRY VARCHAR(16777216),
	DME_FREQUENCY_KHZ FLOAT,
	DME_CHANNEL NUMBER(38,0),
	DME_LATITUDE_DEG FLOAT,
	DME_LONGITUDE_DEG FLOAT,
	DME_ELEVATION_FT NUMBER(38,0),
	SLAVED_VARIATION_DEG FLOAT,
	MAGNETIC_VARIATION_DEG FLOAT,
	USAGE_TYPE VARCHAR(16777216),
	POWER FLOAT,
	ASSOCIATED_AIRPORT VARCHAR(16777216),
	ASSOCIATED_AIRPORT_IDENT VARCHAR(16777216),
	primary key (ID_NAV)
);

create or replace TABLE MY_TEST.NEW.COUNTRIES (
	ID NUMBER(38,0),
	CODE VARCHAR(16777216),
	NAME VARCHAR(16777216),
	CONTINENT VARCHAR(16777216),
	WIKIPEDIA_LINK VARCHAR(16777216),
	KEYWORDS VARCHAR(16777216)
	primary key (ID)
);

create or replace TABLE MY_TEST.NEW.AIRPORT_COMMENTS (
	ID NUMBER(38,0),
	threadRef FLOAT,
	airportRef NUMBER(38,0),
	airportIdent VARCHAR(16777216),
	DATE VARCHAR(16777216),
	memberNickname VARCHAR(16777216),
	SUBJECT VARCHAR(16777216),
	BODY VARCHAR(16777216)\
	PRIMARY KEY (ID)
);


### Views used in snowflake


In [ ]:
airport_per_continent = '''create or replace view AIRPORT_PER_CONTINENT as
    -- comment = '<comment>'
    SELECT continent,    
       COUNT(CASE WHEN type IN ('seaplane_base','medium_airport', 'large_airport') THEN 1 END) AS airports,
       COUNT(CASE WHEN type = 'heliport' THEN 1 END) AS heliport,
       COUNT(CASE WHEN type = 'small_airport' THEN 1 END) AS airfield       
       FROM airports
       GROUP BY continent;'''

airport_per_country = '''create or replace view AIRPORT_PER_COUNTRY as
SELECT iso_country,
/* assumumes the small airport is an airfield*/
       COUNT(CASE WHEN type IN ('seaplane_base','medium_airport', 'large_airport') THEN 1 END) AS airports,
       COUNT(CASE WHEN type = 'heliport' THEN 1 END) AS heliport,
       COUNT(CASE WHEN type = 'small_airport' THEN 1 END) AS airfield       
       FROM airports
       GROUP BY iso_country;'''


In [ ]:
Business questions with PostgreSQL
1. 

SELECT c.name AS country, 
       COUNT(CASE WHEN a.type = 'large_airport' THEN 1 END) AS airports,
       COUNT(CASE WHEN a.type = 'small_airport' THEN 1 END) AS airfields,
       COUNT(CASE WHEN a.type = 'heliport' THEN 1 END) AS heliports
FROM countries c
JOIN regions r ON c.code = r.iso_country
JOIN airports a ON r.code = a.iso_region
GROUP BY c.name;
// BY CONTINENT

SELECT a.continent,
       COUNT(CASE WHEN a.type = 'large_airport' THEN 1 END) AS airports,
       COUNT(CASE WHEN a.type = 'small_airport' THEN 1 END) AS airfields,
       COUNT(CASE WHEN a.type = 'heliport' THEN 1 END) AS heliports
FROM airports a
GROUP BY a.continent;

2. 
SELECT c.name AS country,
       AVG(CASE WHEN a.type = 'large_airport' THEN a.elevation_ft END) AS avg_airport_elevation,
       AVG(CASE WHEN a.type = 'small_airport' THEN a.elevation_ft END) AS avg_airfield_elevation,
       AVG(CASE WHEN a.type = 'heliport' THEN a.elevation_ft END) AS avg_heliport_elevation
FROM countries c
JOIN regions r ON c.code = r.iso_country
JOIN airports a ON r.code = a.iso_region
GROUP BY c.name;
